# Objetives: Parse data from analitcs API into dataframes to generate Reports.
## Report 1. Checkouts on the "Encomenda".
## Report 2. Frequency of search parameters on each Experience/Hotel
### Author: fillipefeitosa <fillipefeitosa@ua.pt>

In [70]:
# List of Imports
import pandas as pd
import re
import csv
import datetime
from requests_html import HTMLSession
from urllib import parse


In [71]:
# Load Data
checkouts = pd.read_excel('pagepath_newusers_FINAL.xlsx')
# paths = pd.read_excel('paths.xlsx')

# Language Control  
canceladoList = ('Storniert', 'Cancelled', 'Annulé', 'Cancelado')
completoList = ['Pagamento Completo', 'Zahlung vollständig', 'Paiement Complet', 'Payment Complete', 'Pago Completo']
bixlink = 'https://bookinxisto.com'
session = HTMLSession()

In [72]:
# checkouts[checkouts.pagePath.str.startswith('/fr')].pagePath[8555]


In [73]:
# url = parse.parse_qs(checkouts.iloc[9464].pagePath)
# url = parse.parse_qsl
# checkouts.head()
# checkouts.loc[checkouts.pagePath.str.startswith('/es')]
# myDict = dict(parse.parse_qs(url))
# myDict['LodgeSearch[date_start]']
# myDict
# checkouts.loc[checkouts.pagePath.str.endswith('order-view')]


In [74]:
# checkouts.loc[checkouts.pagePath.str.startswith('/es')]
# checkouts


In [75]:
# Get results that matches the Language List
# for order in languageList:
#     result = checkouts[checkouts.pageTitle.str.startswith(order)]
# #     print(result)
# checkouts.iloc[9465].pagePath
# for index, row in checkouts.iterrows():
#     url = parse.urlsplit(row.pagePath)
#     print(url)

In [76]:
def getCheckouts(dataframe):
    for index, row in dataframe.iterrows():
        url = parse.urlsplit(row.pagePath)
        
        if(url.path.endswith('order-view')):
            # Get Order Id
            urlParsedParams  = parse.parse_qs(row.pagePath)
            
            if 'id' not in urlParsedParams.keys():
                for key in urlParsedParams.keys():
                    if 'order-view' in key and key.endswith('id'):
                        orderId = urlParsedParams[key][0]
            else:
                orderId = urlParsedParams['id'][0]
            
            # Start Request to verify responsivines
            request = session.get(bixlink+str(row.pagePath))
            
            if(request.status_code == 200):
                dataCSV = []
                # Get All gray buttons with Pagamento and Cancelamentos
                cart_status = request.html.find('.btn--gray')
                # cart_links = list(request.html.links)
                cart_name = request.html.find('.cart__name')
                # Get Cart dates to all orders
                cart_date = request.html.find('.cart__date')
                # Put this guys ordered in the same structure
                zipped = zip(cart_status, cart_name, cart_date)

                for zips in zipped:
                    dataCSV.append(orderId)
                    
                    # Geting Exp/House Name on its Page
                    myR = session.get(list(zips[1].absolute_links)[0])
                    if(myR.status_code==200):
                        expOrHotelName = myR.html.find('.product-title', first=True)
                        dataCSV.append(expOrHotelName.text)  
                    else:
                        dataCSV.append('DESCONTINUADO')

                    # Verify if the order was purchased or canceled
                    dataCSV.append(bixlink+str(row.pagePath))
                    print(bixlink+str(row.pagePath))
                    if(zips[0].text in completoList):
                        dataCSV.append('ok')
                        print('ok')
                    if(zips[0].text in canceladoList):
                        dataCSV.append('cancelado')
                        print('cancelado')
                    
                    # Calculate Number of Nights on order
                    date = zips[2].text.split()
                    print(date)
                    if(len(date)>=2):
                        start_date = date[0]
                        print(start_date)
                        end_date = date[2]
                        print(end_date)
                        finalDate = (datetime.datetime.strptime(end_date, "%d/%m/%y").date() - datetime.datetime.strptime(start_date, "%d/%m/%y").date()).days
                    else:
                        # Some experiences happen only once a day
                        finalDate = 1
                    print(finalDate)
                    dataCSV.append(finalDate)
                    
                    dataCSV.append(zips[0].text)
                    # Add link to the Exp/hotel
                    dataCSV.append(zips[1].absolute_links)
                    with open("checkouts-withDates.csv", "a") as fp:
                        wr = csv.writer(fp, dialect='excel')
                        wr.writerow(dataCSV)
                    # Debug Print and clearing List
                    print(dataCSV)
                    dataCSV = []

In [77]:
getCheckouts(checkouts)

https://bookinxisto.com/de/account/order-view?auth=1KZFsL6xf3A9wYyxm839ra5CTnV59J3i&id=538
cancelado
['22/06/18', 'bis', '24/06/18']
22/06/18
24/06/18
2
['538', 'Casa Cimeira', 'https://bookinxisto.com/de/account/order-view?auth=1KZFsL6xf3A9wYyxm839ra5CTnV59J3i&id=538', 'cancelado', 2, 'Storniert', {'https://bookinxisto.com/de/lodge-catalog/lodge?id=80&LodgeSearch%5Bdate_start%5D=2018-06-22&LodgeSearch%5Bdate_end%5D=2018-06-24&LodgeSearch%5Bnum_adults%5D=6'}]
https://bookinxisto.com/de/account/order-view?auth=2-dUDozJrm7-0xKa4ilzzd10hPSo_Zoj&id=783
ok
['28/08/18', 'bis', '30/08/18']
28/08/18
30/08/18
2
['783', 'Hotel Rural Quinta da Geia', 'https://bookinxisto.com/de/account/order-view?auth=2-dUDozJrm7-0xKa4ilzzd10hPSo_Zoj&id=783', 'ok', 2, 'Zahlung vollständig', {'https://bookinxisto.com/de/lodge-catalog/lodge?id=40&LodgeSearch%5Bdate_start%5D=2018-08-28&LodgeSearch%5Bdate_end%5D=2018-08-30&LodgeSearch%5Bnum_adults%5D=2'}]
https://bookinxisto.com/de/account/order-view?id=539&auth=1KZF

https://bookinxisto.com/en/account/order-view?id=129&auth=E85x5-uBhdql70M5Zft-UK1pBo5Vqovr
cancelado
['30/03/18', 'to', '01/04/18']
30/03/18
01/04/18
2
['129', 'Casa Lausus', 'https://bookinxisto.com/en/account/order-view?id=129&auth=E85x5-uBhdql70M5Zft-UK1pBo5Vqovr', 'cancelado', 2, 'Cancelled', {'https://bookinxisto.com/en/lodge-catalog/lodge?id=57&LodgeSearch%5Bdate_start%5D=2018-03-30&LodgeSearch%5Bdate_end%5D=2018-04-01&LodgeSearch%5Bnum_adults%5D=2'}]
https://bookinxisto.com/en/account/order-view?id=135&auth=SY0UljTXJalkJVtY3BWouAHAU870eYdS
ok
['06/04/18', 'to', '08/04/18']
06/04/18
08/04/18
2
['135', 'Casa da Árvore - Cerdeira - Home for Creativity', 'https://bookinxisto.com/en/account/order-view?id=135&auth=SY0UljTXJalkJVtY3BWouAHAU870eYdS', 'ok', 2, 'Payment Complete', {'https://bookinxisto.com/en/lodge-catalog/lodge?id=19&LodgeSearch%5Bdate_start%5D=2018-04-06&LodgeSearch%5Bdate_end%5D=2018-04-08&LodgeSearch%5Bnum_adults%5D=2'}]
https://bookinxisto.com/en/account/order-view?i

https://bookinxisto.com/en/account/order-view?id=368&auth=zP2G_vKKRx-7ISbEuU8IvKDXH7HuKfeX
ok
['30/03/18', 'to', '01/04/18']
30/03/18
01/04/18
2
['368', 'Casa da Pífara - Aldeia Oliveiras', 'https://bookinxisto.com/en/account/order-view?id=368&auth=zP2G_vKKRx-7ISbEuU8IvKDXH7HuKfeX', 'ok', 2, 'Payment Complete', {'https://bookinxisto.com/en/lodge-catalog/lodge?id=25&LodgeSearch%5Bdate_start%5D=2018-03-30&LodgeSearch%5Bdate_end%5D=2018-04-01&LodgeSearch%5Bnum_adults%5D=2'}]
https://bookinxisto.com/en/account/order-view?id=41&auth=apMl7Tf9_NjV6z26KCbeEILKqD4ZkMWF
cancelado
['15/01/18', 'to', '16/01/18']
15/01/18
16/01/18
1
['41', 'Hotel Parque Serra da Lousã', 'https://bookinxisto.com/en/account/order-view?id=41&auth=apMl7Tf9_NjV6z26KCbeEILKqD4ZkMWF', 'cancelado', 1, 'Cancelled', {'https://bookinxisto.com/en/lodge-catalog/lodge?id=55&LodgeSearch%5Bdate_start%5D=2018-01-15&LodgeSearch%5Bdate_end%5D=2018-01-16&LodgeSearch%5Bnum_adults%5D=2'}]
https://bookinxisto.com/en/account/order-view?id

https://bookinxisto.com/en/account/order-view?id=747&auth=msAcnE8v8xRmlWIhdUH9ccz_1wpQ4uoR
cancelado
['15/08/18', 'to', '18/08/18']
15/08/18
18/08/18
3
['747', 'Casa do Açude - Camping Oleiros', 'https://bookinxisto.com/en/account/order-view?id=747&auth=msAcnE8v8xRmlWIhdUH9ccz_1wpQ4uoR', 'cancelado', 3, 'Cancelled', {'https://bookinxisto.com/en/lodge-catalog/lodge?id=30&LodgeSearch%5Bdate_start%5D=2018-08-15&LodgeSearch%5Bdate_end%5D=2018-08-18&LodgeSearch%5Bnum_adults%5D=4'}]
https://bookinxisto.com/en/account/order-view?id=793&auth=xY0L61agYjvIzvjTUplAb4WBaYKDeN-8
ok
['08/09/18', 'to', '10/09/18']
08/09/18
10/09/18
2
['793', 'Casa da Cancela - Aldeia Oliveiras', 'https://bookinxisto.com/en/account/order-view?id=793&auth=xY0L61agYjvIzvjTUplAb4WBaYKDeN-8', 'ok', 2, 'Payment Complete', {'https://bookinxisto.com/en/lodge-catalog/lodge?id=6&LodgeSearch%5Bdate_start%5D=2018-09-08&LodgeSearch%5Bdate_end%5D=2018-09-10&LodgeSearch%5Bnum_adults%5D=4'}]
https://bookinxisto.com/en/account/order-

https://bookinxisto.com/pt/account/order-view?auth=_tFt52dkDZShRAbHQhDXg_olupuFQ5Oa&id=685
cancelado
['28/07/18', 'a', '30/07/18']
28/07/18
30/07/18
2
['685', 'Casa de Campo da Comareira - Ninho da Coruja', 'https://bookinxisto.com/pt/account/order-view?auth=_tFt52dkDZShRAbHQhDXg_olupuFQ5Oa&id=685', 'cancelado', 2, 'Cancelado', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=47&LodgeSearch%5Bdate_start%5D=2018-07-28&LodgeSearch%5Bdate_end%5D=2018-07-30&LodgeSearch%5Bnum_adults%5D=2'}]
https://bookinxisto.com/pt/account/order-view?auth=-ayuZgYqvGNKr9EuGKACYPrRkx-6Ktli&id=559
cancelado
['13/06/18']
1
['559', 'Restaurante Museu da Chanfana - Menu Degustação', 'https://bookinxisto.com/pt/account/order-view?auth=-ayuZgYqvGNKr9EuGKACYPrRkx-6Ktli&id=559', 'cancelado', 1, 'Cancelado', {'https://bookinxisto.com/pt/experience-catalog/experience?id=150&ExperienceSearch%5Bdate_start%5D=2018-06-11&ExperienceSearch%5Bdate_end%5D=2018-06-15&ExperienceSearch%5Bparticipants%5D=2'}]
https://bookinxi

...

https://bookinxisto.com/pt/account/order-view?id=718&auth=J5AbKsGtYgnyS9qYs20hMbIL4qIhKJ4E
ok
['14/08/18', 'a', '17/08/18']
14/08/18
17/08/18
3
['718', 'Casa da Travessa - Aldeia Oliveiras', 'https://bookinxisto.com/pt/account/order-view?id=718&auth=J5AbKsGtYgnyS9qYs20hMbIL4qIhKJ4E', 'ok', 3, 'Pagamento Completo', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=24&LodgeSearch%5Bdate_start%5D=2018-08-14&LodgeSearch%5Bdate_end%5D=2018-08-17&LodgeSearch%5Bnum_adults%5D=3'}]
https://bookinxisto.com/pt/account/order-view?id=720&auth=1jGWmM4gcZ-IGtt-JlnS60s8X20RLdex
cancelado
['17/08/18', 'a', '20/08/18']
17/08/18
20/08/18
3
['720', 'Casa da Cancela - Aldeia Oliveiras', 'https://bookinxisto.com/pt/account/order-view?id=720&auth=1jGWmM4gcZ-IGtt-JlnS60s8X20RLdex', 'cancelado', 3, 'Cancelado', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=6&LodgeSearch%5Bdate_start%5D=2018-08-17&LodgeSearch%5Bdate_end%5D=2018-08-20&LodgeSearch%5Bnum_adults%5D=4'}]
https://bookinxisto.com/pt/account/or

https://bookinxisto.com/pt/account/order-view?id=759&auth=KZjNGJn0MEwQ1ydp9BOplecds0LVqbsu
cancelado
['04/09/18', 'a', '07/09/18']
04/09/18
07/09/18
3
['759', 'Casa do Açude - Camping Oleiros', 'https://bookinxisto.com/pt/account/order-view?id=759&auth=KZjNGJn0MEwQ1ydp9BOplecds0LVqbsu', 'cancelado', 3, 'Cancelado', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=30&LodgeSearch%5Bdate_start%5D=2018-09-04&LodgeSearch%5Bdate_end%5D=2018-09-07&LodgeSearch%5Bnum_adults%5D=4'}]
https://bookinxisto.com/pt/account/order-view?id=761&auth=Lil-mAXmyCOXqGqdoVUn-xgiKxaQMLFe
cancelado
['17/08/18', 'a', '19/08/18']
17/08/18
19/08/18
2
['761', 'Casas do Rio', 'https://bookinxisto.com/pt/account/order-view?id=761&auth=Lil-mAXmyCOXqGqdoVUn-xgiKxaQMLFe', 'cancelado', 2, 'Cancelado', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=81&LodgeSearch%5Bdate_start%5D=2018-08-17&LodgeSearch%5Bdate_end%5D=2018-08-19&LodgeSearch%5Bnum_adults%5D=2'}]
https://bookinxisto.com/pt/account/order-view?id=762&auth

https://bookinxisto.com/pt/account/order-view?id=792&auth=1GuO07RxmD8NQLYHvBCr9uu23DYSRUUW
cancelado
['07/09/18', 'a', '10/09/18']
07/09/18
10/09/18
3
['792', 'Casa do Açude - Camping Oleiros', 'https://bookinxisto.com/pt/account/order-view?id=792&auth=1GuO07RxmD8NQLYHvBCr9uu23DYSRUUW', 'cancelado', 3, 'Cancelado', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=30&LodgeSearch%5Bdate_start%5D=2018-09-07&LodgeSearch%5Bdate_end%5D=2018-09-10&LodgeSearch%5Bnum_adults%5D=4'}]
https://bookinxisto.com/pt/account/order-view?id=794&auth=zBJ0gQQJoGeO7QpY5t5z22FBl3GXOaLM
cancelado
['22/09/18', 'a', '23/09/18']
22/09/18
23/09/18
1
['794', 'Casa da Ribeira - Aldeia de Camelo', 'https://bookinxisto.com/pt/account/order-view?id=794&auth=zBJ0gQQJoGeO7QpY5t5z22FBl3GXOaLM', 'cancelado', 1, 'Cancelado', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=59&LodgeSearch%5Bdate_start%5D=2018-09-22&LodgeSearch%5Bdate_end%5D=2018-09-23&LodgeSearch%5Bnum_adults%5D=2'}]
https://bookinxisto.com/pt/account/

https://bookinxisto.com/pt/account/order-view?id=829&auth=8LcS6zEZqvXG-PYQvAeLeEIj-VbDlJcy
cancelado
['05/10/18', 'a', '07/10/18']
05/10/18
07/10/18
2
['829', 'DESCONTINUADO', 'https://bookinxisto.com/pt/account/order-view?id=829&auth=8LcS6zEZqvXG-PYQvAeLeEIj-VbDlJcy', 'cancelado', 2, 'Cancelado', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=107&LodgeSearch%5Bdate_start%5D=2018-10-05&LodgeSearch%5Bdate_end%5D=2018-10-07&LodgeSearch%5Bnum_adults%5D=4'}]
https://bookinxisto.com/pt/account/order-view?id=830&auth=8LcS6zEZqvXG-PYQvAeLeEIj-VbDlJcy
ok
['05/10/18', 'a', '07/10/18']
05/10/18
07/10/18
2
['830', 'Casa da Torre - Aldeia de Camelo', 'https://bookinxisto.com/pt/account/order-view?id=830&auth=8LcS6zEZqvXG-PYQvAeLeEIj-VbDlJcy', 'ok', 2, 'Pagamento Completo', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=61&LodgeSearch%5Bdate_start%5D=2018-10-05&LodgeSearch%5Bdate_end%5D=2018-10-07&LodgeSearch%5Bnum_adults%5D=4'}]
https://bookinxisto.com/pt/account/order-view?id=832&auth=7

https://bookinxisto.com/pt/account/order-view?id=98&auth=3V2iGDWjpwi13I4I7KlNOSh4XOxk9OH6
cancelado
['28/04/18', 'a', '30/04/18']
28/04/18
30/04/18
2
['98', 'Aqua Village Health Resort & SPA', 'https://bookinxisto.com/pt/account/order-view?id=98&auth=3V2iGDWjpwi13I4I7KlNOSh4XOxk9OH6', 'cancelado', 2, 'Cancelado', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=96&LodgeSearch%5Bdate_start%5D=2018-04-28&LodgeSearch%5Bdate_end%5D=2018-04-30&LodgeSearch%5Bnum_adults%5D=2'}]
https://bookinxisto.com/pt/account/order-view?id=99&auth=Xdd-V6vQd62RQvoyxZAkq67rHNf6ZSHK
ok
['22/04/18', 'a', '26/04/18']
22/04/18
26/04/18
4
['99', 'Casa do Forno - Vilar dos Condes', 'https://bookinxisto.com/pt/account/order-view?id=99&auth=Xdd-V6vQd62RQvoyxZAkq67rHNf6ZSHK', 'ok', 4, 'Pagamento Completo', {'https://bookinxisto.com/pt/lodge-catalog/lodge?id=63&LodgeSearch%5Bdate_start%5D=2018-04-22&LodgeSearch%5Bdate_end%5D=2018-04-26&LodgeSearch%5Bnum_adults%5D=2'}]


In [78]:
# checkouts.iloc[3]
# for index, row in checkouts.iterrows():
#     url = parse.urlsplit(row.pagePath)
#     print(url)
#     print(url)

In [79]:
r = session.get('https://bookinxisto.com/pt/account/order-view?auth=0RU1eVnhHW9TsXizv3lbA07EwQj43OCc&id=809')

# # r = session.get('https://bookinxisto.com/pt/account/order-view?id=845&auth=TJmxA_fAMNeolGZkhUkgdvZ5g2AC5lLP')
# print(r.status_code)
# cancelados = r.html.find('.btn__text--small')
cart_name = r.html.find('.cart__name')
# cart_line = r.html.find('.cart__line--separator')
# cart_links = r.html.links
cart_data = r.html.find('.cart__date')
# # cart_item = r.html.find('.cart__item')
# print(cancelados[0].text)



start_date = cart_data[0].text.split()[0]
end_date = cart_data[0].text.split()[2]
# daysOrdered = end_date - start_date
end_date

'07/10/18'

In [80]:
temp = datetime.datetime.strptime(start_date, "%d/%m/%y").date() - datetime.datetime.strptime(end_date, "%d/%m/%y").date()

In [81]:
temp.days

-2

In [82]:
# for line in cart_line:
#     cart_name = line.find('.cart_name')
#     cart_status = line.find('.btn--gray')
#     print(cart_status)


In [83]:
# cart_status = r.html.find('.btn--gray')
# cart_name = r.html.find('.cart__name')
# zipped = zip(cart_status, cart_name)
# for zips in zipped:
#     print(zips[0].text + ' - ' + zips[1].text)
#     print(zips[1].absolute_links)
# for status in cart_status:
#     print(status.text)
# for cart in cart_name:
#     print(cart.text)

In [84]:
# for name in cart_name:
#     print(name.text)
#     print(name.absolute_links)

In [85]:
myR = session.get('https://bookinxisto.com/pt/lodge-catalog/lodge?id=63&LodgeSearch%5Bdate_start%5D=2018-04-22&LodgeSearch%5Bdate_end%5D=2018-04-26&LodgeSearch%5Bnum_adults%5D=2')

In [86]:
myR.html.find('.product-title', first=True).text

'Casa do Forno - Vilar dos Condes'